In [38]:
import cv2
from os import listdir
from os.path import join
import numpy as np
import pandas as pd
from sklearn import svm

In [39]:
# Listing all the image files

mypath = 'C:/Users/user/Desktop/Minor/PETA/PETA dataset/CAVIAR4REID/archive'
files = listdir(mypath)
del files[-1]

In [40]:
# Building a list that groups ALL THE VIEWS from DIFFERENT ANGLES of the SAME PERSON

person = []
k = listdir(mypath)

for i in range(len(files)):
    j = '00'+str(i+1)+'_' #To identify all files in the form 1_,2_,3_ etc
    k = '000'+str(i+1)+'_' #To identify all files in the form 1_,2_,3_ etc
    person_num = [f.startswith((j,k)) for f in files] #Boolean values of all places very j(or '(i+1)_') is True
    idx =  [k for k, x in enumerate(person_num) if x] #Return indices of where the values are True
    (person.append([files[l] for l in idx]))
person = [x for x in person if x != []]
# person

In [41]:
#Counting the number of views of each person

mypath = 'C:/Users/user/Desktop/Minor/PETA/PETA dataset/CAVIAR4REID/archive/'
count = np.empty((len(person)),dtype=object)
for i in range(len(person)):
    count[i] = len(person[i])

In [42]:
# Specifying path for each image

person = np.array(person)
person_path = []
mypath = 'C:/Users/user/Desktop/Minor/PETA/PETA dataset/CAVIAR4REID/archive/'
for p in person:
    each_p = p
    for i in each_p:
        i = mypath + i
        (person_path).append(i)

In [43]:
# person_path

In [44]:
# Reading and resizing images

images = np.empty((len(person_path)), dtype=object)
resized_imgs = np.empty((len(person_path)), dtype=object)
for n in range(0, len(person_path)):
    images[n] = cv2.imread(person_path[n],0) #Reading images in grayscale
    resized_imgs[n] = cv2.resize(images[n],(32,32)) #Resizing images to 32x32 standard size
# images

In [45]:
# Importing cleaned labels from csv file

labels = pd.read_csv(filepath_or_buffer = r'C:\Users\user\Desktop\Minor\labelled_caviar.csv',index_col = 0)

In [46]:
labels.head()

,Upper Body Colour,Lower Body Colour,Hair Colour,Footwear Colour,Lower Body Type,Lower Body Clothing,Gender,Age Limit,Upper Body Type,Upper Body Clothing,Hair Length,Footear Type,carrying,Accessories
1,9,0,0,0,0,3,1,2,0,0,0,2,0,0
2,0,0,0,0,0,0,0,0,1,1,0,0,1,0
3,0,0,0,0,1,0,0,1,1,2,0,1,2,0
4,1,1,0,0,0,0,0,1,1,3,0,1,2,0
5,5,2,0,0,0,1,0,1,1,4,0,0,2,0


In [47]:
# Expanding labels to match with the number of images according to count

extended_labels = []
for i in range(len(labels)):
    extended_labels.append(np.tile(np.array(pd.DataFrame(labels.iloc[i,:]).transpose()),(count[i],1)))
    #Copying labels as per the count values
# extended_labels

In [48]:
# Associating each label with each training sample

final_labels = []
for i in range(len(extended_labels)):
    for j in range(len(extended_labels[i])):
            final_labels.append(extended_labels[i][j])
# csv = pd.DataFrame(final_labels)
# csv.to_excel(r'C:\Users\user\Desktop\Labels.xlsx')
# final_labels

In [49]:
# Defining X_train and y_train

mypath = 'C:/Users/user/Desktop/Minor/PETA/PETA dataset/CAVIAR4REID/archive'
k = listdir(mypath)
images = np.empty((len(k)-1),dtype=object)
resized_imgs = np.empty((len(k)-1),dtype=object)
for n in range(0, len(k)-1):
    images[n] = cv2.imread(join(mypath,k[n]),0) #Reading images in grayscale
    resized_imgs[n] = cv2.resize(images[n],(32,32)) #Resizing images to 32x32 standard size
for i in range(0,len(k)-1):
    # Normalizing raw pixel values using mean and standard deviation
    a=resized_imgs[i].mean()
    b=resized_imgs[i].std()
    t=np.ndarray([32,32])
    t.fill(1)
    t=t*a
    resized_imgs[i]=(resized_imgs[i]-a)/b
    
X_train = resized_imgs
y_train = final_labels

In [50]:
# Flattening X_train

for i in range(len(k)-1):
    X_train[i] = X_train[i].flatten().tolist()

In [51]:
#Flattening y_train

y_train = np.array(y_train)
for i in range(len(k)-1):
    y_train[i] = y_train[i].flatten().tolist()

In [52]:
X_train = np.array(list(X_train), dtype=np.float)

In [53]:
%%time
# Training an SVM

trained_SVM = []
clf=svm.SVC(gamma=0.001)
m,n = y_train.shape

Wall time: 2 ms


In [41]:
# TESTING

In [42]:
# Reading X_test and normalizing

testpath = 'C:/Users/user/Desktop/Minor/Test Set'
l = listdir(testpath)
test_images = np.empty((len(l)-1),dtype=object)
test_resized_imgs = np.empty((len(l)-1),dtype=object)
for n in range(0, len(l)-1):
    test_images[n] = cv2.imread(join(testpath,l[n]),0) #Reading images in grayscale
    test_resized_imgs[n] = cv2.resize(test_images[n],(32,32)) #Resizing images to 32x32 standard size
for i in range(0,len(l)-1):
    # Normalizing raw pixel values using mean and standard deviation
    mean=test_resized_imgs[i].mean()
    std=test_resized_imgs[i].std()
    t=np.ndarray([32,32])
    t.fill(1)
    t=t*mean
    test_resized_imgs[i]=(test_resized_imgs[i]-mean)/std
    
X_test = test_resized_imgs

In [43]:
# Flattening X_test

X_test = np.array(X_test)

for i in range(len(X_test)):
    X_test[i] = X_test[i].flatten().tolist()
X_test = np.array(list(X_test), dtype=np.float)

In [44]:
%%time
# Predicting

predictions = np.empty((n,len(X_test)))
for i in range(len(X_test)):
    for j in range(0,n):
        clf.fit(X_train,y_train[:,j])
        predictions[j][i] = clf.predict(X_test[i])

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C

Wall time: 7min 9s


C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [45]:
predictions

array([[ 1.,  0.,  0.,  1.,  1.,  0.,  0.,  2.,  0.,  0.,  0.,  1.,  1.,
         0.,  0.],
       [ 0.,  2.,  2.,  2.,  2.,  2.,  0.,  5.,  2.,  2.,  2.,  2.,  2.,
         2.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.],
       [ 1.,  1.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,
         1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.],
       [ 6.,  6.,  4.,  1.,  6.,  4.,  6.,  3.,  3.,  3.,  4.,  6.,  3.,
         4.,  6.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
       

In [ ]:
# KNN

In [138]:
%%time
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
knn.fit(X_train, y_train)
print(knn.predict(X_test))

[[0 0 0 0 1 0 1 1 0 1 0 0 0 0]
 [2 2 0 0 0 0 0 0 1 4 0 3 2 0]
 [0 5 0 0 0 0 1 0 1 3 0 0 2 0]
 [1 2 0 0 1 1 0 1 1 1 0 0 2 0]
 [1 0 0 0 1 0 1 1 0 1 0 0 2 0]
 [0 2 0 0 1 2 0 1 0 9 0 1 2 0]
 [0 0 0 0 0 0 0 1 1 1 0 0 2 0]
 [2 5 0 0 0 0 0 1 1 4 0 3 2 0]
 [0 5 0 0 0 0 0 1 1 4 0 0 2 0]
 [0 2 0 1 0 1 1 1 1 6 1 0 2 0]
 [0 0 0 0 0 0 0 1 1 3 0 0 2 0]
 [1 2 0 0 1 2 0 1 0 1 0 1 2 0]
 [1 2 0 0 0 0 0 1 1 1 0 1 2 0]
 [0 2 0 0 0 1 0 1 1 9 0 0 2 0]
 [0 0 0 0 0 1 0 1 1 4 0 0 2 0]]
Wall time: 133 ms
